## Face recognition Project

##### PT:
Importação das bibliotecas necessárias para o projeto.
TensorFlow será usado para construir e treinar o modelo de aprendizado profundo.
MobileNetV2 é um modelo pré-treinado utilizado para Transfer Learning.
ImageDataGenerator é usado para pré-processar e aumentar os dados de entrada.

##### ENG:
Import the necessary libraries for the project.
TensorFlow is used to build and train the deep learning model.
MobileNetV2 is a pre-trained model used for Transfer Learning.
ImageDataGenerator is used for preprocessing and augmenting input data.

### 1. Configuração do Dataset / Dataset Config

*PT:*
- Define os caminhos para os datasets de treinamento e validação.
- Cria geradores de dados que aplicam Data Augmentation nas imagens de treinamento.
- O Data Augmentation inclui normalização e transformações para melhorar a generalização do modelo.
- As imagens são redimensionadas para 224x224 para compatibilidade com o MobileNetV2.


*ENG:*
- Define paths for training and validation datasets.
- Create data generators that apply Data Augmentation to training images.
- Data Augmentation includes normalization and transformations to improve model generalization.
- Images are resized to 224x224 to be compatible with MobileNetV2.

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Carregar o modelo pré-treinado sem a última camada
base_model = MobileNetV2(weights='imagenet', include_top=False)

# Adicionar camadas personalizadas no topo
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)  # Duas classes: Hugh Jackman e Ryan Reynolds

model = Model(inputs=base_model.input, outputs=predictions)

# Congelar camadas base
for layer in base_model.layers:
    layer.trainable = False


C:\Users\Kaique\AppData\Local\Temp\ipykernel_7200\2221646573.py:9: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


### 2. Carregar o modelo pré-treinado / Load the Pre_trained model

PT:
- Carrega o modelo MobileNetV2 pré-treinado na ImageNet sem as camadas de classificação originais.
- Adiciona camadas personalizadas para adaptar o modelo ao problema de reconhecimento facial.
- O modelo base é congelado para preservar os pesos originais durante o treinamento inicial.

ENG:
- Load the pre-trained MobileNetV2 model from ImageNet without the original classification layers.
- Add custom layers to adapt the model to the face recognition problem.
- The base model is frozen to preserve the original weights during the initial training phase.


In [2]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Separar validação
)

train_generator = train_datagen.flow_from_directory(
    'dataset/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'dataset/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 251 images belonging to 2 classes.
Found 61 images belonging to 2 classes.


### 3. Compilar e treinar modelo / Compile and Train the model

PT:
- Compila o modelo com o otimizador Adam, uma taxa de aprendizado pequena e a função de perda categorical_crossentropy.
- Treina o modelo usando os geradores de treinamento e validação por 10 épocas.

ENG:
- Compile the model using the Adam optimizer, a small learning rate, and the categorical_crossentropy loss function.
- Train the model using the training and validation generators for 10 epochs.


In [6]:
# Compilar o modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,  # Ajuste conforme necessário
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# Salvar o modelo
model.save('face_recognition_model.h5')


C:\Users\Kaique\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 746ms/step - accuracy: 0.4600 - loss: 2.1313

C:\Users\Kaique\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.4664 - loss: 2.1402 - val_accuracy: 0.9375 - val_loss: 0.1505
Epoch 2/10
1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 466ms/step - accuracy: 0.7188 - loss: 0.6207

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2032.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - accuracy: 0.7188 - loss: 0.6207 - val_accuracy: 0.8966 - val_loss: 0.1548
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 772ms/step - accuracy: 0.7649 - loss: 0.7720
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 151ms/step - accuracy: 0.8438 - loss: 0.4933 - val_accuracy: 0.6562 - val_loss: 0.7265
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 832ms/step - accuracy: 0.7982 - loss: 0.4918 - val_accuracy: 0.9310 - val_loss: 0.1692
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8889 - loss: 0.2536 
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 892ms/step - accuracy: 0.8332 - loss: 0.3550 - val_accuracy: 0.8750 - val_loss: 0.2811
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - accuracy: 0.8750 - loss: 0.2947 - val_accuracy: 0.9310 - val_loss: 0.1642
Epoch 9/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 753ms/step - accuracy: 0.8960 - loss: 0.2277
Epoch 10/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 152ms/step - accuracy: 0.9375 - loss: 0.1735 - val_accuracy: 0.9688 - val_loss: 0.1345


### 4. Salvando o modelo e Fazendo Reconhecimento / Saving the model and Making Previsions


PT:
- Salva o modelo treinado no disco para ser reutilizado no reconhecimento facial.


ENG:
- Save the trained model to disk for reuse in face recognition.


In [8]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Carregar o modelo salvo
model = load_model('face_recognition_model.h5')

# Carregar e processar uma imagem
image = cv2.imread('test_Huge_Rey.jpg')
image_resized = cv2.resize(image, (224, 224))
image_array = np.expand_dims(image_resized, axis=0) / 255.0

# Fazer predição
predictions = model.predict(image_array)
class_indices = train_generator.class_indices
classes = {v: k for k, v in class_indices.items()}

predicted_class = classes[np.argmax(predictions)]
confidence = np.max(predictions)

print(f"Predicted: {predicted_class} with confidence {confidence:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted: Ryan_Reynolds with confidence 0.76


### 5. Reconhecimento Facial na Imagem / Facial Recognition on images

PT:
- Carrega o modelo salvo para realizar o reconhecimento facial.
- Pré-processa a imagem fornecida para corresponder ao formato de entrada do modelo.
- Realiza predições e utiliza coordenadas fornecidas pela face_detection para desenhar retângulos ao redor dos rostos.
- Exibe graficamente os resultados indicando o nome da pessoa reconhecida.



ENG:
- Load the saved model to perform face recognition.
- Preprocess the given image to match the model's input format.
- Make predictions and use coordinates provided by face_detection to draw rectangles around faces.
- Display results graphically, showing the recognized person's name.


In [10]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Carregar o modelo treinado
model = load_model('face_recognition_model.h5')

# Carregar o classificador de rostos Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Dicionário de classes
class_indices = {'Hugh_Jackman': 0, 'Ryan_Reynolds': 1}
classes = {v: k for k, v in class_indices.items()}

# Carregar a imagem
image = cv2.imread('test_Huge_Rey.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detectar rostos
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

# Loop pelos rostos detectados
for (x, y, w, h) in faces:
    # Recortar o rosto
    face = image[y:y+h, x:x+w]
    face_resized = cv2.resize(face, (224, 224))
    face_array = np.expand_dims(face_resized, axis=0) / 255.0

    # Fazer predição
    predictions = model.predict(face_array)
    predicted_class = classes[np.argmax(predictions)]
    confidence = np.max(predictions)

    # Adicionar rótulo na imagem
    label = f"{predicted_class} ({confidence*100:.2f}%)"
    color = (0, 255, 0) if predicted_class == 'Hugh_Jackman' else (255, 0, 0)
    cv2.rectangle(image, (x, y), (x+w, y+h), color, 2)
    cv2.putText(image, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

# Exibir a imagem
cv2.imshow('Detected Faces', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
